In [1]:
%matplotlib notebook
import glob
import pandas as pd
import mne
import matplotlib.pyplot as plt
from io import StringIO
import mne
from mne.io import read_raw_eeglab, read_epochs_eeglab
import numpy as np
from scipy import signal
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm_notebook
from sklearn.svm import SVR

import autosklearn.regression
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import multiprocessing
from tpot import TPOTRegressor
from oct2py import octave

from joblib import Parallel, delayed
import multiprocessing
from joblib import wrap_non_picklable_objects
import json
import pickle
import os.path
from mpl_toolkits.mplot3d import axes3d
from mayavi import mlab

/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/pyparsing.py:2927: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
eeglab_path = '/home/raquib/Documents/MATLAB/eeglab2019_0/functions/'
octave.addpath(eeglab_path + 'guifunc');
octave.addpath(eeglab_path + 'popfunc');
octave.addpath(eeglab_path + 'adminfunc');
octave.addpath(eeglab_path + 'sigprocfunc');
octave.addpath(eeglab_path + 'miscfunc');

In [3]:
experiment = 'data/original/*/*'
meps = sorted(glob.glob(experiment + '/mep/*/*.txt'))
mep_present = len(meps) > 0
eegs = sorted(glob.glob(experiment + '/eeg/*/clean-prestimulus.set'))
eeg_present = len(eegs) > 0
cmaps = sorted(glob.glob(experiment + '/cmap/*.xlsx'))
cmap_present = len(cmaps) > 0
all_present = mep_present and eeg_present and cmap_present
print(all_present)

True


In [4]:
print('EEG count: ' + str(len(eegs)))
print('MEP count: ' + str(len(meps)))
print('CMAP count: ' + str(len(cmaps)))

EEG count: 43
MEP count: 77
CMAP count: 22


In [5]:
eegs

['data/original/sub03/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub03/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
 'data/original/sub03/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
 'data/original/sub03/exp02/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub03/exp02/eeg/SP 110RMT r2/clean-prestimulus.set',
 'data/original/sub03/exp03/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub03/exp03/eeg/SP 110RMT r2/clean-prestimulus.set',
 'data/original/sub03/exp03/eeg/SP 110RMT r3/clean-prestimulus.set',
 'data/original/sub04/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub04/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
 'data/original/sub04/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
 'data/original/sub05/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
 'data/original/sub06/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub06/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
 'data/original/sub06/exp01/eeg/SP

In [6]:
eegs = [
    'data/original/sub03/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub03/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub03/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
#     'data/original/sub03/exp02/eeg/SP 110RMT r1/clean-prestimulus.set', NO CMAP
#     'data/original/sub03/exp02/eeg/SP 110RMT r2/clean-prestimulus.set', NO CMAP
#     'data/original/sub03/exp03/eeg/SP 110RMT r1/clean-prestimulus.set', NO CMAP
    'data/original/sub03/exp03/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub03/exp03/eeg/SP 110RMT r3/clean-prestimulus.set',
#     'data/original/sub04/exp01/eeg/SP 110RMT r1/clean-prestimulus.set', NO CMAP
#     'data/original/sub04/exp01/eeg/SP 110RMT r2/clean-prestimulus.set', NO CMAP
#     'data/original/sub04/exp01/eeg/SP 110RMT r3/clean-prestimulus.set', NO CMAP
#     'data/original/sub05/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub06/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub06/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub06/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub06/exp02/eeg/SP 110RMT/clean-prestimulus.set',
    'data/original/sub07/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub07/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub07/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub08/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub08/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub08/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub08/exp02/eeg/SP 110RMT/clean-prestimulus.set',
    'data/original/sub08/exp03/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub08/exp03/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub08/exp03/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub12/exp02/eeg/SP 110RMT/clean-prestimulus.set',
    'data/original/sub13/exp01/eeg/SP 110RMT/clean-prestimulus.set',
    'data/original/sub14/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub15/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub15/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub15/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub16/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub16/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub16/exp01/eeg/SP 110RMT r3/clean-prestimulus.set'
]

# Read and process EEG

In [7]:
def read_eeg(path):
    eeg = octave.pop_loadset(path)
    new_trial_list = []
    for i in range(eeg.data.shape[2]):
        trial = eeg.data[:, :, i]
        time = np.linspace(-1000, -20, num=trial.shape[1])
        trial = pd.DataFrame(np.transpose(trial), columns=eeg.chanlocs.labels[0])
        trial['time'] = time
        new_trial_list.append(trial)
    return new_trial_list

def crop_trials(trial_list, duration_millis=500, sampling_rate=2048):
    new_trial_list = []
    for trial in trial_list:
        samples_to_pick = duration_millis * sampling_rate / 1000
        new_trial_list.append(trial.tail(int(samples_to_pick)))
    return new_trial_list, samples_to_pick

# Calculate EEG area.
def calculate_eeg_area(epoch_df, sf=2048):
    y = epoch_df.drop('time', axis=1).mean(axis=1)
    b2, a2 = signal.butter(4, 200/(sf/2), btype='lowpass')
    envelope = signal.filtfilt(b2, a2, np.abs(y))
    area = np.trapz(envelope, epoch_df['time'].values)
    return area

# Calculate EEG frequency.
def calculate_eeg_frequency(channel):
    sf = 2048
    win = 4 * sf
    freqs, psd = signal.welch(channel, sf, nperseg=win)
    return freqs, psd

def calculate_eeg_max_amplitude(epoch_df):
    avg = epoch_df.mean(axis=1)
    return np.max(avg.values)

def band_max(freq, psd, start=0, end=4):
    indices = np.where((freq >= start) & (freq <= end))
    freq = freq[indices]
    psd = psd[indices]
    i = np.argmax(np.abs(psd))
    return freq[i], psd[i]

def filter_electrodes(trial, which='all'):
    time_column = trial['time']
    if which == 'ltm1':
        channel_names = ['FC5','FC1','C3','CP5','CP1','FC3','C5','C1','CP3']
    elif which == 'rtm1':
        channel_names = ['FC6','FC2','C4','CP6','CP2','FC4','C6','C2','CP4']
    elif which == 'central':
        channel_names = ['Fz','FCz','Cz','F1','FC1','C1','C2','FC2','F2']
    else:
        channel_names = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'O2', 'EOG', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CP4', 'P5', 'P1', 'P2', 'P6', 'PO5', 'PO3', 'PO4', 'PO6', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Oz']
    trial = trial[channel_names]
    trial['time'] = time_column
    return trial

def read_wavelets(eeg_path, epoch_num):
    segments = eeg_path.split('/')
    path = 'wavelets/' + segments[2] + '-' + segments[3] + '-' + segments[5] + '-' + str(epoch_num)
    with open(path + '-central.pickle', 'rb') as f:
        central = pickle.load(f)
    with open(path + '-ltm1.pickle', 'rb') as f:
        ltm1 = pickle.load(f)
    with open(path + '-rtm1.pickle', 'rb') as f:
        rtm1 = pickle.load(f)
    with open(path + '-all.pickle', 'rb') as f:
        all_channels = pickle.load(f)
    return all_channels, ltm1, rtm1, central

# Read and process MEP

In [8]:
def get_mep_path(eeg_path):
    segments = eeg_path.split('/')
    segments[4] = 'mep'
    segments = segments[:-1]
    mep_path = '/'.join(segments) + '/*.txt'
    mep_path = glob.glob(mep_path)[0]
    return mep_path

def get_cmap_path(eeg_path):
    segments = eeg_path.split('/')
    segments[4] = 'cmap'
    del segments[5]
    segments = segments[:-1]
    cmap_path = '/'.join(segments) + '/*.xlsx'
    cmap_paths = glob.glob(cmap_path)
    if (len(cmap_paths) == 0):
        print('No CMAP found for ' + cmap_path)
        return ''
    return cmap_paths[0]

def mep_area(mep):
    mep, _ = crop_mep_region(mep)
    b2, a2 = signal.butter(4, 250/(3750/2), btype='lowpass')
    emg_envelope = signal.filtfilt(b2, a2, np.abs(mep[get_apb_column_name(mep)]))
    return np.trapz(np.abs(emg_envelope), np.linspace(1, mep.shape[0], mep.shape[0]))

def calculate_mep_categories(mep_frames):
    sizes = []
    for mep_frame in mep_frames:
        sizes.append(calculate_mep_size(mep_frame))
    p1 = np.percentile(sizes, 33.33)
    p2 = np.percentile(sizes, 66.66)
    cat = np.ones(len(mep_frames)) * ((sizes >= p1) & (sizes < p2))
    cat = 2 * (sizes >= p2) * np.ones(len(mep_frames)) + cat
    return cat

def calculate_mep_categories_binary(mep_frames):
    sizes = []
    for mep_frame in mep_frames:
        sizes.append(calculate_mep_size(mep_frame))
    p1 = np.percentile(sizes, 50)
    cat = np.ones(len(mep_frames)) * (sizes >= p1)
    return cat

def calculate_mep_categories_cmap(mep_frames, cmap):
    cmap_max, cmap_min = find_cmap_peaks(cmap)
    cmap = cmap_max - cmap_min
    sizes = []
    for mep_frame in mep_frames:
        sizes.append(calculate_mep_size(mep_frame) / cmap)
    p1 = np.percentile(sizes, 33.33)
    p2 = np.percentile(sizes, 66.66)
    cat = np.ones(len(mep_frames)) * ((sizes >= p1) & (sizes < p2))
    cat = 2 * (sizes >= p2) * np.ones(len(mep_frames)) + cat
    return cat

def calculate_mep_categories_cmap_binary(mep_frames, cmap):
    cmap_max, cmap_min = find_cmap_peaks(cmap)
    cmap = cmap_max - cmap_min
    sizes = []
    for mep_frame in mep_frames:
        sizes.append(calculate_mep_size(mep_frame) / cmap)
    p1 = np.percentile(sizes, 50)
    cat = np.ones(len(mep_frames)) * (sizes >= p1)
    return cat

# Calculate MEP size.
def calculate_mep_size(mep_frame):
    mep_cropped, time = crop_mep_region(mep_frame)
    apb_name = get_apb_column_name(mep_frame)
    max_row = mep_frame.iloc[mep_cropped.idxmax(axis=0)[apb_name]]
    min_row = mep_frame.iloc[mep_cropped.idxmin(axis=0)[apb_name]]
    mep_size = max_row[apb_name] - min_row[apb_name]
    return mep_size

# Function to crop MEP region.
def crop_mep_region(mep_frame):
    crop_start = 0.211
    crop_end = 0.4
    multiplier = 7499 / 1.4998
    start = int(crop_start * multiplier)
    end = int(crop_end * multiplier)
    mep_cropped = mep_frame.iloc[start:end, :]
    time = mep_frame.iloc[start:end, :]
    return mep_cropped, time

# Function to get APB column name.
def get_apb_column_name(mep_frame):
    if 'L APB' in mep_frame:
        return 'L APB'
    else:
        return 'APB'

# Open MEP file as dataframe.
def open_mep_as_df(path):
    fileMep = open(path, "r+")
    mep_frames = fileMep.read().split('\n\n')
    df_list = []
    for mep_frame in mep_frames:
        df_list.append(pd.read_csv(StringIO(mep_frame), '\t'))
    return df_list

def get_mep_category_absolute(mep_size):
    if mep_size <= .2:
        return 0
    elif mep_size > .2 and mep_size < 1:
        return 1
    else:
        return 2
    
def get_mep_category_absolute_binary(mep_size):
    if mep_size <= .5:
        return 0
    else:
        return 1

# Read and process CMAP

In [9]:
# Read CMAP file.
def read_cmap(path):
    df = pd.read_excel(path, sheet_name='Raw data', header=None)
    cmap = df.iloc[44:df.shape[0]-1, 38:59]
    time = df.iloc[44:df.shape[0]-1, 0]
    df = pd.DataFrame(cmap)
    df['time'] = time
    return df

# Identify CMAP peaks.
def find_cmap_peaks(cmap):
    time = cmap['time']
    mean_clean = cmap.drop('time', axis=1).mean(axis=1)
    mean_clean[0:104*10-1] = 0
    idxmax = mean_clean.idxmax(axis=0)
    idxmax = time[idxmax]
    idxmin = mean_clean.idxmin(axis=0)
    idxmin = time[idxmin]
    max_val = mean_clean.max()
    min_val = mean_clean.min()
    return max_val, min_val

# Extract all features from EEG, CMAP and MEP

In [10]:
def extract_features(eeg_path):
    mep_path = get_mep_path(eeg_path)
    epochs = read_eeg(eeg_path)
    epochs, _ = crop_trials(epochs)
    mep_frames = open_mep_as_df(mep_path)
    cmap_path = get_cmap_path(eeg_path)
    cmap = read_cmap(cmap_path)
    epoch_features = []
    if len(epochs) != len(mep_frames):
        raise Exception('MEP frame count (' + str(len(mep_frames)) + ') is not equal to EEG epochs count (' + str(len(epochs)) + ') for ' + eeg_path + ' and ' + mep_path)
        return
    mep_sizes = calculate_mep_categories(mep_frames)
    mep_sizes_binary = calculate_mep_categories_binary(mep_frames)
    mep_sizes_cmap = calculate_mep_categories_cmap(mep_frames, cmap)
    mep_sizes_cmap_binary = calculate_mep_categories_cmap_binary(mep_frames, cmap)
    for i in range(len(epochs)):
        epoch = epochs[i]
        mep = mep_frames[i]
        segments = eeg_path.split('/')
        d = {'sub': segments[2], 'exp': segments[3], 'run': segments[5], 'epoch': i}

        freq, psd = calculate_eeg_frequency(epoch.drop('time', axis=1).values.mean(axis=1))
        d['all_all_max_power_freq'], d['all_all_max_power'] = band_max(freq, psd, 4, 40)
        d['all_delta_max_power_freq'], d['all_delta_max_power'] = band_max(freq, psd, 0, 4)
        d['all_theta_max_power_freq'], d['all_theta_max_power'] = band_max(freq, psd, 4, 8)
        d['all_alpha_max_power_freq'], d['all_alpha_max_power'] = band_max(freq, psd, 8, 16)
        d['all_beta_max_power_freq'], d['all_beta_max_power'] = band_max(freq, psd, 16, 32)
        d['all_gamma_max_power_freq'], d['all_gamma_max_power'] = band_max(freq, psd, 32, 500)

        freq, psd = calculate_eeg_frequency(filter_electrodes(epoch, 'ltm1').drop('time', axis=1).values.mean(axis=1))
        d['ltm1_all_max_power_freq'], d['ltm1_all_max_power'] = band_max(freq, psd, 4, 40)
        d['ltm1_delta_max_power_freq'], d['ltm1_delta_max_power'] = band_max(freq, psd, 0, 4)
        d['ltm1_theta_max_power_freq'], d['ltm1_theta_max_power'] = band_max(freq, psd, 4, 8)
        d['ltm1_alpha_max_power_freq'], d['ltm1_alpha_max_power'] = band_max(freq, psd, 8, 16)
        d['ltm1_beta_max_power_freq'], d['ltm1_beta_max_power'] = band_max(freq, psd, 16, 32)
        d['ltm1_gamma_max_power_freq'], d['ltm1_gamma_max_power'] = band_max(freq, psd, 32, 500)

        freq, psd = calculate_eeg_frequency(filter_electrodes(epoch, 'rtm1').drop('time', axis=1).values.mean(axis=1))
        d['rtm1_all_max_power_freq'], d['rtm1_all_max_power'] = band_max(freq, psd, 4, 40)
        d['rtm1_delta_max_power_freq'], d['rtm1_delta_max_power'] = band_max(freq, psd, 0, 4)
        d['rtm1_theta_max_power_freq'], d['rtm1_theta_max_power'] = band_max(freq, psd, 4, 8)
        d['rtm1_alpha_max_power_freq'], d['rtm1_alpha_max_power'] = band_max(freq, psd, 8, 16)
        d['rtm1_beta_max_power_freq'], d['rtm1_beta_max_power'] = band_max(freq, psd, 16, 32)
        d['rtm1_gamma_max_power_freq'], d['rtm1_gamma_max_power'] = band_max(freq, psd, 32, 500)

        freq, psd = calculate_eeg_frequency(filter_electrodes(epoch, 'central').drop('time', axis=1).values.mean(axis=1))
        d['central_all_max_power_freq'], d['central_all_max_power'] = band_max(freq, psd, 4, 40)
        d['central_delta_max_power_freq'], d['central_delta_max_power'] = band_max(freq, psd, 0, 4)
        d['central_theta_max_power_freq'], d['central_theta_max_power'] = band_max(freq, psd, 4, 8)
        d['central_alpha_max_power_freq'], d['central_alpha_max_power'] = band_max(freq, psd, 8, 16)
        d['central_beta_max_power_freq'], d['central_beta_max_power'] = band_max(freq, psd, 16, 32)
        d['central_gamma_max_power_freq'], d['central_gamma_max_power'] = band_max(freq, psd, 32, 500)
        
        # Wavelet
        wt_all, wt_ltm1, wt_rtm1, wt_central = read_wavelets(eeg_path, i)
        
        # Drop last 9 rows (freq 0 to 2Hz)
        wt_all = wt_all.drop(wt_all.tail(21).index)
        wt_ltm1 = wt_ltm1.drop(wt_ltm1.tail(21).index)
        wt_rtm1 = wt_rtm1.drop(wt_all.tail(21).index)
        wt_central = wt_central.drop(wt_all.tail(21).index)
        
        d['wavelet_all_max_power_at_freq'] = wt_all.mean(axis=1).argmax() * 1000
        d['wavelet_all_max_power_freq'] = wt_all.mean(axis=1).max()
        d['wavelet_ltm1_max_power_at_freq'] = wt_ltm1.mean(axis=1).argmax() * 1000
        d['wavelet_ltm1_max_power_freq'] = wt_ltm1.mean(axis=1).max()
        d['wavelet_rtm1_max_power_at_freq'] = wt_rtm1.mean(axis=1).argmax() * 1000
        d['wavelet_rtm1_max_power_freq'] = wt_rtm1.mean(axis=1).max()
        d['wavelet_central_max_power_at_freq'] = wt_central.mean(axis=1).argmax() * 1000
        d['wavelet_central_max_power_freq'] = wt_central.mean(axis=1).max()
        
        d['wavelet_all_max_power_at_time'] = wt_all.mean(axis=0).argmax()
        d['wavelet_all_max_power_time'] = wt_all.mean(axis=0).max()
        d['wavelet_ltm1_max_power_at_time'] = wt_ltm1.mean(axis=0).argmax()
        d['wavelet_ltm1_max_power_time'] = wt_ltm1.mean(axis=0).max()
        d['wavelet_rtm1_max_power_at_time'] = wt_rtm1.mean(axis=0).argmax()
        d['wavelet_rtm1_max_power_time'] = wt_rtm1.mean(axis=0).max()
        d['wavelet_central_max_power_at_time'] = wt_central.mean(axis=0).argmax()
        d['wavelet_central_max_power_time'] = wt_central.mean(axis=0).max()

        d['area_under_eeg'] = calculate_eeg_area(epoch)
        d['rtm1_area_under_eeg'] = calculate_eeg_area(filter_electrodes(epoch, 'rtm1'))
        d['ltm1_area_under_eeg'] = calculate_eeg_area(filter_electrodes(epoch, 'ltm1'))
        d['central_area_under_eeg'] = calculate_eeg_area(filter_electrodes(epoch, 'central'))
        
        d['all_all_max_amplitude'] = calculate_eeg_max_amplitude(epoch.drop('time', axis=1))
        d['rtm1_all_max_amplitude'] = calculate_eeg_max_amplitude(filter_electrodes(epoch, 'rtm1').drop('time', axis=1))
        d['ltm1_all_max_amplitude'] = calculate_eeg_max_amplitude(filter_electrodes(epoch, 'ltm1').drop('time', axis=1))
        d['central_all_max_amplitude'] = calculate_eeg_max_amplitude(filter_electrodes(epoch, 'central').drop('time', axis=1))

        d['all_alpha_max_amplitude'] = calculate_eeg_max_amplitude(epoch.drop('time', axis=1))
        d['rtm1_alpha_max_amplitude'] = calculate_eeg_max_amplitude(filter_electrodes(epoch, 'rtm1').drop('time', axis=1))
        d['ltm1_alpha_max_amplitude'] = calculate_eeg_max_amplitude(filter_electrodes(epoch, 'ltm1').drop('time', axis=1))
        d['central_alpha_max_amplitude'] = calculate_eeg_max_amplitude(filter_electrodes(epoch, 'central').drop('time', axis=1))
        
        d['cmap_max'], d['cmap_min'] = find_cmap_peaks(cmap)
        
        mep_size = calculate_mep_size(mep)
        d['mep_size'] = mep_size
        d['mep_category_percentile'] = mep_sizes[i]
        d['mep_category_binary'] = mep_sizes_binary[i]
        d['mep_category_absolute'] = get_mep_category_absolute(mep_size)
        d['mep_category_absolute_binary'] = get_mep_category_absolute_binary(mep_size)
        d['mep_category_cmap'] = mep_size / (d['cmap_max'] - d['cmap_min'])
        d['mep_category_cmap_category'] = mep_sizes_cmap[i]
        d['mep_category_cmap_category_binary'] = mep_sizes_cmap_binary[i]
        d['mep_area'] = mep_area(mep)
        
        epoch_features.append(d)
    return epoch_features

In [11]:
def extract_and_save_features(eeg_path, filename):
    features = extract_features(eeg_path)
    features = pd.DataFrame(features)
    if os.path.isfile(filename):
        old_features = pd.read_excel(filename, index_col=0)
        features = old_features.append(features)
    features.to_excel(filename)

In [12]:
features_filename = '47-features-v3.xlsx'

In [13]:
for eeg_path in tqdm_notebook(eegs):
    extract_and_save_features(eeg_path, features_filename)

pop_loadset(): loading file data/original/sub03/exp01/eeg/SP 110RMT r1/clean-prestimulus.set ...
Reading float file 'data/original/sub03/exp01/eeg/SP 110RMT r1/clean-prestimulus.fdt'...
Scaling components to RMS microvolt


/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/scipy/signal/spectral.py:1969: UserWarning: nperseg = 8192 is greater than input length  = 1024, using nperseg = 1024
  .format(nperseg, input_length))
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/ipykernel_launcher.py:63: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
/home/raquib/anaconda3/envs/tmseeg/lib/pyt

pop_loadset(): loading file data/original/sub03/exp01/eeg/SP 110RMT r2/clean-prestimulus.set ...
Reading float file 'data/original/sub03/exp01/eeg/SP 110RMT r2/clean-prestimulus.fdt'...
Scaling components to RMS microvolt
pop_loadset(): loading file data/original/sub03/exp01/eeg/SP 110RMT r3/clean-prestimulus.set ...
Reading float file 'data/original/sub03/exp01/eeg/SP 110RMT r3/clean-prestimulus.fdt'...
Scaling components to RMS microvolt
pop_loadset(): loading file data/original/sub03/exp03/eeg/SP 110RMT r2/clean-prestimulus.set ...
Reading float file 'data/original/sub03/exp03/eeg/SP 110RMT r2/clean-prestimulus.fdt'...
Scaling components to RMS microvolt
pop_loadset(): loading file data/original/sub03/exp03/eeg/SP 110RMT r3/clean-prestimulus.set ...
Reading float file 'data/original/sub03/exp03/eeg/SP 110RMT r3/clean-prestimulus.fdt'...
Scaling components to RMS microvolt
pop_loadset(): loading file data/original/sub06/exp01/eeg/SP 110RMT r1/clean-prestimulus.set ...
Reading float f